In [1]:
from fundar.structures import List as Lista, slicer
from fundar.utils import (
    snd, 
    load_from_str_or_buf, 
    has,
    identity, 
    access, 
    identity, 
    is_empty, 
    find_file, 
    split_path_recursively
)

from pandas import DataFrame
from data_transformers.consumer import parse_source, transformer_from_source, str_split, bytes_decode
from fundar import json
from data_transformers import chain
from pandas import DataFrame
from glob import glob
import os

In [2]:
file = find_file('mappings.json', 'C:/Users/Joan/Coding/Python/data_transformers')

In [3]:
t_1 = find_file('CRECIM_g01_transformer.py', '../../../')

In [4]:
mappings = find_file('mappings.json', t_1)

In [5]:
datasets_update = '/'.join(split_path_recursively(t_1)[:-3])
datasets_update_files = glob(datasets_update+'/*.csv')
transformers_files = glob(datasets_update+'/definitivos/transformers/*.py')
transformers_files = Lista(transformers_files)

In [6]:
get_name = (
    os.path.split,  # list[str]
    snd,            # str
    str_split('_'), # list[str]
    slicer(2),      # list[str]
    '_'.join        # str 
) # str

get_name = Lista.of(Lista(get_name).chain, lambda x: x) # (str, str)

name_mappings = {k:v for k,v in transformers_files.map(get_name.apply)}

In [7]:
files = {y : {'file':k, 'transformer': t } 
 for k,v in json.load(mappings).items() 
 for x in v 
 for y in (x['public'], ) # alias
 for t in (name_mappings.get(y, None), ) # alias
 if t is not None # filtro los que no tengan transformadores
}

In [8]:
source_code = load_from_str_or_buf(files['CRECIM_g01']['transformer']).readlines()
source_code = Lista(source_code).map(bytes_decode('latin-1'))

In [9]:
pipeline_str, f_definitions = parse_source(source_code)

In [12]:
defined_functions = Lista(f_definitions).map(transformer_from_source)

for k,v in defined_functions:
    globals()[k] = v

In [13]:
pipeline = eval(pipeline_str)

In [14]:
df = DataFrame({'pais_nombre': ['b'], 'sector': ['a']})

In [15]:
pipeline(df)

([(<transformer 'start'>,
   {},
     pais_nombre sector
   0           b      a),
  (<transformer 'drop_col'>,
   {'col': 'pais_nombre',
    'axis': 1,
    'df':   pais_nombre sector
    0           b      a},
     sector
   0      a),
  (<transformer 'identity'>,
   {'df':   sector
    0      a},
     sector
   0      a)],
   sector
 0      a)